In [1]:
pip install datasets torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
!pip install imblearn

In [3]:
import pandas as pd
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score

# Read in data
df = pd.read_csv('/content/drive/MyDrive/reviews_with_sentiment.csv')
df = df.head(10000)
print(df.head())


   top_id            date_modified  \
0  115447  2024-05-20 15:04:36.770   
1  115447  2024-05-20 15:04:36.770   
2  115447  2024-05-20 15:04:36.770   
3  115447  2024-05-20 15:04:36.770   
4  115447  2024-05-20 15:04:36.770   

                                                text  sentiment  
0  popped in here randomly on a sunday to get a b...          1  
1  everything wa delicious  even my husband  who ...          1  
2  the food is fresh  modern  and flavoursome  th...          1  
3  it ’ s a nice place to have a week end brunch ...          1  
4  nice scottish breakfast but service a little s...          1  


In [4]:
df['sentiment'].value_counts()

sentiment
1    8224
2    1254
0     522
Name: count, dtype: int64

In [5]:
from imblearn.over_sampling import RandomOverSampler
import numpy as np

ros = RandomOverSampler()
train_x, train_y = ros.fit_resample(np.array(df['text']).reshape(-1, 1), np.array(df['sentiment']).reshape(-1, 1));
train_os = pd.DataFrame(list(zip([x[0] for x in train_x], train_y)), columns = ['text', 'sentiment']);
train_os['sentiment'].value_counts()

sentiment
1    8224
0    8224
2    8224
Name: count, dtype: int64

In [6]:
X = train_os['text'].values
y = train_os['sentiment'].values

In [7]:
# Split the data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    X, y, test_size=0.3, random_state=42)

train_texts = list(map(str, train_texts))
test_texts = list(map(str, test_texts))
# Convert train_labels to a Pandas Series to use value_counts()
train_labels_series = pd.Series(train_labels)
print(train_labels_series.value_counts())

1    5762
0    5761
2    5747
Name: count, dtype: int64


In [9]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3) # Specify num_labels for binary classification

# Tokenize the datasets
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')

#No need to convert to numpy array. train_labels is already a numpy array
train_labels_onehot = enc.fit_transform(train_labels.reshape(-1, 1)).toarray()
test_labels_onehot = enc.transform(test_labels.reshape(-1, 1)).toarray()

# Convert one-hot encoded labels to PyTorch tensors
train_labels = torch.tensor(train_labels_onehot, dtype=torch.float32)  # Use float32 for BCEWithLogitsLoss
test_labels = torch.tensor(test_labels_onehot, dtype=torch.float32)

# Prepare datasets for Trainer
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})

test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': test_labels
})

# Set up Trainer for fine-tuning
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Updated from deprecated 'evaluation_strategy'
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Ensure the model is on the correct device (e.g., GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Fine-tune the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TrainOutput(global_step=6477, training_loss=0.2017035800144042, metrics={'train_runtime': 1440.4355, 'train_samples_per_second': 35.968, 'train_steps_per_second': 4.497, 'total_flos': 3407976543121920.0, 'train_loss': 0.2017035800144042, 'epoch': 3.0})

In [10]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Evaluate the model
preds = trainer.predict(test_dataset)

# Get predicted class labels
predictions = preds.predictions.argmax(-1)  # Get the index of the highest logit (predicted class)

# Convert test_labels (tensor) to numpy array for compatibility with sklearn metrics
test_labels_np = test_labels.cpu().numpy().argmax(-1)  # Convert one-hot encoded labels back to class labels

# Calculate accuracy and F1 score
accuracy = accuracy_score(test_labels_np, predictions)
f1 = f1_score(test_labels_np, predictions, average='weighted')

# Print the accuracy and F1 score
print(f"Accuracy: {accuracy * 100:.2f}, F1 Score: {f1}")

# Generate and print the classification report
target_names = ['Negative', 'Neutral', 'Positive']  # Adjust these names based on your actual labels
print(classification_report(test_labels_np, predictions, target_names=target_names))

Accuracy: 96.24, F1 Score: 0.9622961461139473
              precision    recall  f1-score   support

    Negative       0.99      1.00      1.00      2463
     Neutral       0.98      0.91      0.94      2462
    Positive       0.92      0.98      0.95      2477

    accuracy                           0.96      7402
   macro avg       0.96      0.96      0.96      7402
weighted avg       0.96      0.96      0.96      7402



In [11]:
# Import the necessary function
from sklearn.metrics import confusion_matrix
# Import the necessary libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Generate the confusion matrix
conf_matrix = confusion_matrix(test_labels_np, predictions)
#show confustion matrix
fig, ax =plt.subplots(figsize=(5,5))
labels=['Negative', 'Neutral', 'Positive']
ax=sns.heatmap(conf_matrix, annot=True, cmap="Blues", fmt='g', cbar=False, annot_kws={"size":25})
plt.title('BERT Sentiment Analysis\nConfusion Matrix', fontsize=15)
ax.xaxis.set_ticklabels(labels, fontsize=17)
ax.yaxis.set_ticklabels(labels, fontsize=17)
ax.set_ylabel('Test', fontsize=20)
ax.set_xlabel('Predicted', fontsize=20)
plt.show()